In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

model_data = pd.read_parquet('Leobrix_Release_BBILA_model_Data.parquet.gzip')
#Merge with new one banking features
sal_data = pd.read_csv('stated_salary_vs_actual_salary.csv')
sal_data = sal_data[['latest_feature_id','diff_btw_stated_salary_vs_actual_salary']]
model_data = pd.merge(model_data, sal_data, how='left',on='latest_feature_id')
#Merge with new pg features
pg_data= pd.read_parquet('s3://sandbox-payments-data-transfer/cashloans/Uptop_Darwin_24/feat_old_pipeline/pg_features/pg_features.parquet')
model_data = pd.merge(model_data, pg_data, how='left',on='latest_feature_id')
print('********* Data loaded ********* ')

********* Required Libraries Imported *********
********* Data loaded ********* 


In [2]:
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)
# Create one feature from inututive bureau features
data['num_CONSUMER LOAN_enq_last_6M_12M'] = (data['num_CONSUMER LOAN_enq_last_6M']+data['num_CONSUMER LOAN_enq_last_36M'])/2

In [3]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2852) (12128, 2852)


In [176]:
features_list =[
           
'distinct_merchant',
'SUM(raw.amount WHERE action = 1)/SUM(raw.amount)#ever',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'NUM_UNIQUE(pg_success_raw_data.month_end_start_category)',
 'ENTROPY(raw.mode_cleaned)#720',
'max_one_time_spent',
'amt_first_android_txn',
'MAX(pg_success_raw_data.amount)',
'top_payment_mode_count',
'amt_first_cc_txn',
'STD(pg_success_raw_data.amount)',
'month_year',
'90_in_24mob'

              
 ]




In [177]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_2806694/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_2806694/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [178]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [179]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 11), (15604, 11), (23406,), (15604,))

In [180]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 11) (4682, 11) (15604, 11)


In [181]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)

In [182]:
# params = {
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc',
#     'learning_rate': 0.01,
#      'max_depth': 2,

#     'subsample': 0.4,
#     'colsample_bytree': 0.4,
#     # 'min_child_weight':25,

#     #'lambda': 2.6,
#     'lambda': 5.5,
#     'alpha':5.0,
#     'gamma':4.5,
#     # 'scale_pos_weight':3.0,
#     'random_state': 42
    
# }
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 1,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 7,
    'alpha':5,
    'gamma':5,
   
     # 'scale_pos_weight':3.2,
     'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 100,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.55380	eval-auc:0.54293
[10]	train-auc:0.57021	eval-auc:0.56234
[20]	train-auc:0.57503	eval-auc:0.56590
[30]	train-auc:0.57540	eval-auc:0.56733
[40]	train-auc:0.57597	eval-auc:0.56794
[50]	train-auc:0.57641	eval-auc:0.56849
[60]	train-auc:0.57652	eval-auc:0.56876
[70]	train-auc:0.57698	eval-auc:0.56903
[80]	train-auc:0.57703	eval-auc:0.56893
[90]	train-auc:0.57725	eval-auc:0.56888
[99]	train-auc:0.57743	eval-auc:0.56893


In [183]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.58, Gini: 0.15, KS: 0.12
Eval AUC: 0.57, Gini: 0.14, KS: 0.12
Test AUC: 0.54, Gini: 0.08, KS: 0.07
OOT AUC: 0.56, Gini: 0.12, KS: 0.10


In [189]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.217771         0.209025        0.209863        0.201632   5616   
7          0.177324         0.190031        0.201609        0.182717   1861   
6          0.168017         0.178024        0.182710        0.175846   1851   
5          0.152105         0.174655        0.175841        0.173432   1900   
4          0.153436         0.172082        0.173414        0.169877   1877   
3          0.160619         0.167259        0.169872        0.165633   1874   
2          0.122703         0.164635        0.165628        0.163558   1687   
1          0.097668         0.163042        0.163510        0.162846   2058   

        label_good  label_bad  
Decile                         
8             4393       1223  
7             1531        330  
6             1540        311  
5             1611        289  
4             1589        288  
3             1573        301  
2             1480        207  
1             1857        201

In [190]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.198462         0.209050        0.209863        0.201558   4681   
7          0.176810         0.189761        0.201548        0.182379   1561   
6          0.171154         0.177718        0.182379        0.175795   1560   
5          0.167657         0.174582        0.175786        0.173454   1515   
4          0.169365         0.172132        0.173414        0.169872   1606   
3          0.150386         0.167271        0.169869        0.165670   1556   
2          0.141536         0.164664        0.165640        0.163536   1406   
1          0.137289         0.163036        0.163510        0.162846   1719   

        label_good  label_bad  
Decile                         
8             3752        929  
7             1285        276  
6             1293        267  
5             1261        254  
4             1334        272  
3             1322        234  
2             1207        199  
1             1483        236

In [191]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.208414         0.209176        0.209863        0.204575   3637   
7          0.187964         0.192917        0.204546        0.184123   1213   
6          0.171334         0.179280        0.184094        0.176683   1214   
5          0.178512         0.175517        0.176672        0.174350   1210   
4          0.173663         0.173146        0.174344        0.171618   1215   
3          0.135678         0.168904        0.171593        0.166478   1194   
2          0.131494         0.165306        0.166473        0.163953   1232   
1          0.118714         0.163126        0.163948        0.162846   1213   

        label_good  label_bad  
Decile                         
8             2879        758  
7              985        228  
6             1006        208  
5              994        216  
4             1004        211  
3             1032        162  
2             1070        162  
1             1069        144

In [192]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
6                    MAX(pg_success_raw_data.amount)        23.0
2  MEAN(raw.amount WHERE device_os_category = Win...        20.0
4                                 max_one_time_spent        18.0
9                    STD(pg_success_raw_data.amount)        13.0
1  SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...         8.0
7                             top_payment_mode_count         5.0
5                              amt_first_android_txn         5.0
8                                   amt_first_cc_txn         4.0
3  NUM_UNIQUE(pg_success_raw_data.month_end_start...         3.0
0                                  distinct_merchant         1.0


In [193]:
feature_importance_df

Feature  Importance
6                    MAX(pg_success_raw_data.amount)        23.0
2  MEAN(raw.amount WHERE device_os_category = Win...        20.0
4                                 max_one_time_spent        18.0
9                    STD(pg_success_raw_data.amount)        13.0
1  SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...         8.0
7                             top_payment_mode_count         5.0
5                              amt_first_android_txn         5.0
8                                   amt_first_cc_txn         4.0
3  NUM_UNIQUE(pg_success_raw_data.month_end_start...         3.0
0                                  distinct_merchant         1.0

In [194]:
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_PG_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_PG_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_PG_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_PG_Oot_Pred_9024.csv',index=False)


In [196]:
xgb_model.save_model('LeoPayu_PG_90_in_24mob.json')
print('Model Saved')

Model Saved
